In [7]:
import os
cache_dir = "/scratches/dialfs/alta/hln35/.cache"
os.environ['TRANSFORMERS_CACHE'] = '/scratches/dialfs/alta/hln35/.cache'

In [8]:
model_small = "google/flan-t5-small"

In [20]:
from datasets import load_dataset

data_points = load_dataset("SetFit/rte", split='train', cache_dir=cache_dir)

Found cached dataset json (/scratches/dialfs/alta/hln35/.cache/SetFit___json/SetFit--rte-2cc6a4d81c8aa68d/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


In [21]:
data_points

Dataset({
    features: ['text1', 'text2', 'label', 'idx', 'label_text'],
    num_rows: 2490
})

In [22]:
data_points[0]

{'text1': 'No Weapons of Mass Destruction Found in Iraq Yet.',
 'text2': 'Weapons of Mass Destruction Found in Iraq.',
 'label': 1,
 'idx': 0,
 'label_text': 'not entailment'}

In [26]:
prefix = "rule: for each question there are 2 paragraph in context. answer the question by saying whether they are entailment by return 1 or not return 0. "


In [15]:
max_input_length = 1024

In [16]:
from transformers import AutoTokenizer
import torch
from transformers import AutoModelForQuestionAnswering, AutoModel, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained(model_small)
model = AutoModelForSeq2SeqLM.from_pretrained(model_small)
model1 = AutoModel.from_pretrained(model_small)
model2 = AutoModelForQuestionAnswering.from_pretrained(model_small)



Some weights of T5ForQuestionAnswering were not initialized from the model checkpoint at google/flan-t5-small and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
model


T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo): 

In [10]:
model1

T5Model(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo): Linear(in_features=

In [11]:
model2

T5ForQuestionAnswering(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo): Line

In [17]:
import json, re

In [33]:
# for i in range(len(data_points)):
correct = 0
empty = 0
pattern = r'<pad>\s+(.*?)</s>'  
wrong_outputs = []

for i in range(len(data_points)):
    text1 = data_points[i]["text1"]
    text2 = data_points[i]["text2"]
    ans = ""
    text = prefix + "context: paragraph 1: " + text1 + ". paragraph 2: " + text2 + "."
    ref = str(data_points[i]["label"])
    inputs = tokenizer(text, max_length=max_input_length, truncation=True, return_tensors="pt").input_ids
    preds_tokenized = model.generate(inputs, max_new_tokens=128, do_sample=False) 
    preds = tokenizer.batch_decode(preds_tokenized)

    match = re.search(pattern, preds[0]) 

    if match:
        ans = match.group(1)
    
    if ans == ref:
        correct += 1
    elif ans not in ["0", "1"]:
        empty += 1
        wrong_outputs.append(preds[0])
    # print(text)
    # print(preds)
    # print(ans)
    # print(ref)
# import json
# with open("translate_bleu_small.txt", "w") as fp:
#     json.dump(scores, fp)
# print(sum(scores)/len(data_points))

In [34]:
print(f"Total points are {len(data_points)}. Number of correct answers is {correct}. Number of empty answers is {empty}")

Total points are 2490. Number of correct answers is 1220. Number of empty answers is 1


In [35]:
wrong_outputs

['<pad> no</s>']

In [36]:
model_large = "google/flan-t5-large"
tokenizer_large = AutoTokenizer.from_pretrained(model_large)
model_large = AutoModelForSeq2SeqLM.from_pretrained(model_large)

In [37]:
correct = 0
empty = 0
pattern = r'<pad>\s+(.*?)</s>'  
wrong_outputs = []

for i in range(len(data_points)):
    text1 = data_points[i]["text1"]
    text2 = data_points[i]["text2"]
    ans = ""
    text = prefix + "context: paragraph 1: " + text1 + ". paragraph 2: " + text2 + "."
    ref = str(data_points[i]["label"])
    inputs = tokenizer_large(text, max_length=max_input_length, truncation=True, return_tensors="pt").input_ids
    preds_tokenized = model_large.generate(inputs, max_new_tokens=128, do_sample=False) 
    preds = tokenizer_large.batch_decode(preds_tokenized)

    match = re.search(pattern, preds[0]) 

    if match:
        ans = match.group(1)
    
    if ans == ref:
        correct += 1
    elif ans not in ["0", "1"]:
        empty += 1
        wrong_outputs.append(preds[0])

In [38]:
print(f"Total points are {len(data_points)}. Number of correct answers is {correct}. Number of empty answers is {empty}")

Total points are 2490. Number of correct answers is 296. Number of empty answers is 0
